# Nonlinear Classification and Regression with Decision Trees

### Decision Trees
* Tree-like graphs that model a decision
  * Learned by recursively splitting the set of training instances into subsets based on instances features
  * Nodes are connected by edges that specify possible outcomes of the tests
  * Children nodes test their subsets until reaching a stoppoing criterion
  * Classification tasks - leaf nodes represent classes
  * Regression tasks - values of the response variable produce the estimate
* Making a prediction requires following edges until a leaf node is reached

### Creating a Decision Tree with Iterative Dichotomiser 3 (ID3)

**Example:** You are tasked with classifying animals as cats or dogs. 
However, you cannot observe the animals directly, but instead must rely
on only whether the animal likes the following attributes:
* Playing fetch
* Frequently grumpy
* Favorite of three types of food  

To classify new animals, examine a feature at each node in the decision tree.

Tests that reduce entropy (uncertainty about the data) the most are best done first. 
In the example this may be whether the animal likes playing fetch since almost
all dogs like fetch while almost no cats enjoy it.

Entropy is given by: $H(X) = -\sum_{i=1}^{n}P(x_i) log_2 P(x_i)$ **,**  
where n is the number of outcomes and $P(x_i)$ is the probability of outcome *i*

##### Information Gain -
the difference between the entropy of the parent node and $H(T)$ and the weighted
sum of the childrens' entropies. Higher information gain means a more desirable
test. ID3 breaks ties by choosing one set arbitrarily.

##### Gini Impurity -
Measures the proportions of classes in a set: 
$Gini(t) = 1 - \sum_{i=1}^{j}P(i|t)^2$ **,**  
where *j* is the number of classes, *t* is the subset of instances for the node, 
and *P(i|t)* is the probability of selecting an element of class *i* from the
node subset. Gini impurity is equal to zero when all the elements of the set 
are the same class. It is greatest when each class has an equal proability of
being seleted.

$Gini_max = 1 - \frac{1}{n}$ **,** where n is the number of possible classes.

data for the problem below can be found
[here](http://archive.ics.uci.edu/ml/datasets/Internet+Advertisements).


In [3]:
# Using decision trees to block banner ads on web pages by
# predicting whether an image is an ad or article content
# Explanitory variables: Dimensions of the image,
#   words from page's URL, words from images URL, image's alt text,
#   image's anchor text, window of words surrounding image tag,
#   and dimensions of the image.
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV

df = pd.read_csv('./ad.data', header=None)

explanatory_variable_columns = set(df.columns.values)
explanatory_variable_columns.remove(len(df.columns.values)-1)
response_variable_column = df[len(df.columns.values)-1] # Last column describes the classes

# Encode advertisements as the positive class (1) and content as the 
# negative class (0). Missing values marked with ' *?' and replace with -1
y = [1 if e == 'ad.' else 0 for e in response_variable_column]
X = df[list(explanatory_variable_columns)].copy()
X.replace(to_replace=' *?', value=-1, regex=True, inplace=True)
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Create pipeline and instance of DecisionTreeClassifier
# Set criterion to entropy to build tree using Information Gain
pipeline = Pipeline([
    ('clf', DecisionTreeClassifier(criterion='entropy'))
])
parameters = {
    'clf__max_depth': (150, 155, 160),
    'clf__min_samples_split': (2, 3),
    'clf__min_samples_leaf': (1, 2, 3)
}

# Set grid search to optimise the model's f1 score
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, scoring='f1')
grid_search.fit(X_train, y_train)

best_parameters = grid_search.best_estimator_.get_params()
print('Best score: %0.3f' % grid_search.best_score_)
print('Best parameters set:')
for param_name in sorted(parameters.keys()):
    print('t%s: %r' % (param_name, best_parameters[param_name]))
    
predictions = grid_search.predict(X_test)
print(classification_report(y_test, predictions))

/home/will/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Fitting 3 folds for each of 18 candidates, totalling 54 fits


[Parallel(n_jobs=-1)]: Done  54 out of  54 | elapsed:    9.2s finished


Best score: 0.889
Best parameters set:
tclf__max_depth: 155
tclf__min_samples_leaf: 1
tclf__min_samples_split: 3
             precision    recall  f1-score   support

          0       0.98      0.98      0.98       720
          1       0.87      0.89      0.88       100

avg / total       0.97      0.97      0.97       820



### Advantages and Disadvantages of Decision Trees
* Advantages
  * Easy to use and do not require the data to be standardized
  * Can tolerate missing values for features (although not yet in scikit-learn)
  * Can learn to ignore irrelevant features
  * Can learn to determine which features are most useful
  * Supports multi-output tasks
  * Small decision tree can be easy to visualize
  * Single decision tree - renders multi-class classification
* Disadvantages
  * More prone to overfitting than many other models
  * Learning algorithms can produce large, complicated decision trees
  * Perfectly models every training instance, but fails to generalize the real relationship
  * Pruning removes some of the tallest nodes and leaves (but not supported in scikit-learn)
    * Can do pre-pruning by setting max depth for tree
    * Creating child nodes only when training instances exceeds a threshold
  * Other strategies for getting around complicated trees:
    * Create multiple decision trees
    
#### ID3
* Efficient decision tree learning algorithms are greedy
* Learns efficiently by making locally optimal decisions
* Constructs a tree by selecting a sequence of features to test
* Feature selected reduces the uncertainty of the node
* Locally sub-optimal tests may be required